**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Dec 12 14:01:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   computer.vbs  'My Drive'   NAPRAVE	 yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15376, done.
remote: Total 15376 (delta 0), reused 0 (delta 0), pack-reused 15376
Receiving objects: 100% (15376/15376), 13.98 MiB | 16.33 MiB/s, done.
Resolving deltas: 100% (10341/10341), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:

# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_custom.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_custom.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_custom.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_custom.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_custom.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_custom.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_custom.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_custom.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_custom.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_custom.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-12-12 14:24:44--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  50.4MB/s    in 3.1s    

2021-12-12 14:24:47 (50.4 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/0244.txt       
  inflating: data/obj/0245.png       
  inflating: data/obj/0245.txt       
  inflating: data/obj/0246.png       
  inflating: data/obj/0246.txt       
  inflating: data/obj/0247.png       
  inflating: data/obj/0247.txt       
  inflating: data/obj/0248.png       
  inflating: data/obj/0248.txt       
  inflating: data/obj/0249.png       
  inflating: data/obj/0249.txt       
  inflating: data/obj/0250.png       
  inflating: data/obj/0250.txt       
  inflating: data/obj/0001.png       
  inflating: data/obj/0001.txt       
  inflating: data/obj/0002.png       
  inflating: data/obj/0002.txt       
  inflating: data/obj/0003.png       
  inflating: data/obj/0003.txt       
  inflating: data/obj/0004.png       
  inflating: data/obj/0004.txt       
  inflating: data/obj/0005.png       
  inflating: data/obj/0005.txt       
  inflating: data/obj/0006.png       
  inflating: data/obj/0006.txt       
  inflating: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.5333333333333333 0.35833333333333334 0.05416666666666667 0.1388888888888889 

['0', '0.5333333333333333', '0.35833333333333334', '0.05416666666666667', '0.1388888888888889']
0 data/obj/0105.txt
0 0.5333333333333333 0.35833333333333334 0.05416666666666667 0.1388888888888889
0 0.7989583333333333 0.55 0.11041666666666666 0.23333333333333334 

['0', '0.7989583333333333', '0.55', '0.11041666666666666', '0.23333333333333334']
1 data/obj/0056.txt
0 0.7989583333333333 0.55 0.11041666666666666 0.23333333333333334
0 0.20625 0.4736111111111111 0.041666666666666664 0.24166666666666667 

['0', '0.20625', '0.4736111111111111', '0.041666666666666664', '0.24166666666666667']
1 data/obj/0056.txt
0 0.20625 0.4736111111111111 0.041666666666666664 0.24166666666666667
0 0.23020833333333332 0.4583333333333333 0.08958333333333333 0.16111111111111112 

['0', '0.23020833333333332', '0.4583333333333333', '0.08958333333333333', '0.16111111111111112']
2 data/obj/0118.txt
0 0.23020833333333332 0.45833333333333

In [ ]:
import glob
images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/0182.png', 'data/obj/0101.png', 'data/obj/0071.png', 'data/obj/0037.png', 'data/obj/0169.png', 'data/obj/0248.png', 'data/obj/0190.png', 'data/obj/0061.png', 'data/obj/0024.png', 'data/obj/0226.png', 'data/obj/0148.png', 'data/obj/0013.png', 'data/obj/0080.png', 'data/obj/0200.png', 'data/obj/0005.png', 'data/obj/0040.png', 'data/obj/0131.png', 'data/obj/0246.png', 'data/obj/0003.png', 'data/obj/0168.png', 'data/obj/0077.png', 'data/obj/0072.png', 'data/obj/0062.png', 'data/obj/0019.png', 'data/obj/0151.png', 'data/obj/0045.png', 'data/obj/0095.png', 'data/obj/0025.png', 'data/obj/0214.png', 'data/obj/0017.png', 'data/obj/0048.png', 'data/obj/0018.png', 'data/obj/0014.png', 'data/obj/0078.png', 'data/obj/0244.png', 'data/obj/0177.png', 'data/obj/0208.png', 'data/obj/0004.png', 'data/obj/0240.png', 'data/obj/0145.png', 'data/obj/0051.png', 'data/obj/0046.png', 'data/obj/0057.png', 'data/obj/0065.png', 'data/obj/0012.png', 'data/obj/0041.png', 'data/obj/0067.png', 'data/obj/00

In [ ]:
!ls data/obj

0001.png  0032.txt  0064.png  0095.txt	0127.png  0158.txt  0190.png  0221.txt
0001.txt  0033.png  0064.txt  0096.png	0127.txt  0159.png  0190.txt  0222.png
0002.png  0033.txt  0065.png  0096.txt	0128.png  0159.txt  0191.png  0222.txt
0002.txt  0034.png  0065.txt  0097.png	0128.txt  0160.png  0191.txt  0223.png
0003.png  0034.txt  0066.png  0097.txt	0129.png  0160.txt  0192.png  0223.txt
0003.txt  0035.png  0066.txt  0098.png	0129.txt  0161.png  0192.txt  0224.png
0004.png  0035.txt  0067.png  0098.txt	0130.png  0161.txt  0193.png  0224.txt
0004.txt  0036.png  0067.txt  0099.png	0130.txt  0162.png  0193.txt  0225.png
0005.png  0036.txt  0068.png  0099.txt	0131.png  0162.txt  0194.png  0225.txt
0005.txt  0037.png  0068.txt  0100.png	0131.txt  0163.png  0194.txt  0226.png
0006.png  0037.txt  0069.png  0100.txt	0132.png  0163.txt  0195.png  0226.txt
0006.txt  0038.png  0069.txt  0101.png	0132.txt  0164.png  0195.txt  0227.png
0007.png  0038.txt  0070.png  0101.txt	0133.png  0164.txt  0196.

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

Izpis pretočnega predvajanja je skrajšan na toliko zadnjih vrstic: 5000.
 total_bbox = 68456, rewritten_bbox = 0.000000 % 

 3926: 0.100957, 0.094458 avg loss, 0.001000 rate, 5.996227 seconds, 62816 images, 0.183716 hours left
Loaded: 0.000068 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.003390, iou_loss = 0.000000, total_loss = 0.003390 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.789594), count: 2, class_loss = 1.117065, iou_loss = 0.151306, total_loss = 1.268370 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.001020, iou_loss = 0.000000, total_loss = 0.001020 
 total_bbox = 68458, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: